In [11]:
import pandas as pd
import mysql.connector
import re
conn = mysql.connector.connect(username= "root",
                               host = "127.0.0.1",
                               password = "Ananthu@123",
                               database = "data_spark")

cursor = conn.cursor()

In [12]:
create_sales_table='''CREATE TABLE sales (
    order_number INT,
    line_item INT NOT NULL,
    order_date DATE NOT NULL,
    delivery_date DATE,
    customerkey INT NOT NULL,
    storekey INT NOT NULL,
    productkey INT NOT NULL,
    quantity INT NOT NULL,
    currency_code VARCHAR(10),
    shipping_days INT
);'''
cursor.execute(create_sales_table)
conn.commit()

In [13]:
df_sales = pd.read_csv('Sales.csv')

In [14]:
df_sales.columns = df_sales.columns.str.lower()
df_sales.columns = df_sales.columns.str.replace(" ","_")
df_sales.columns
df_sales['order_date']= pd.to_datetime(df_sales['order_date'])
df_sales['delivery_date']= pd.to_datetime(df_sales['delivery_date'])

In [15]:
df_sales["shipping_days"] = (df_sales['delivery_date'] - df_sales['order_date']).dt.days

In [17]:
df_sales['delivery_date'] = df_sales['delivery_date'].fillna((df_sales['order_date'] + pd.Timedelta(days=4)))

In [18]:
df_sales["shipping_days"] = (df_sales['delivery_date'] - df_sales['order_date']).dt.days

In [19]:
df_sales.head()

,order_number,line_item,order_date,delivery_date,customerkey,storekey,productkey,quantity,currency_code,shipping_days
0,366000,1,2016-01-01,2016-01-05,265598,10,1304,1,CAD,4
1,366001,1,2016-01-01,2016-01-13,1269051,0,1048,2,USD,12
2,366001,2,2016-01-01,2016-01-13,1269051,0,2007,1,USD,12
3,366002,1,2016-01-01,2016-01-12,266019,0,1106,7,CAD,11
4,366002,2,2016-01-01,2016-01-12,266019,0,373,1,CAD,11


In [20]:
import pymysql  # or your preferred MySQL library
import pandas as pd

# Sample DataFrame
# df_stores = pd.DataFrame({
#     'storekey': [...],
#     'country': [...],
#     'state': [...],
#     'square_meters': [...],
#     'open_date': [...]
# })

# Establish connection to the database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',           # Corrected from 'username' to 'user'
    password='Ananthu@123',
    database='data_spark'
)
cursor = conn.cursor()
insert_sales_values = '''
    INSERT INTO sales (
        order_number, 
        line_item, 
        order_date, 
        delivery_date, 
        customerkey, 
        storekey, 
        productkey, 
        quantity, 
        currency_code, 
        shipping_days
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''

# Insert data into the sales table
for index, row in df_sales.iterrows():
    cursor.execute(insert_sales_values, (
        row['order_number'],
        row['line_item'],
        row['order_date'],
        row['delivery_date'],
        row['customerkey'],
        row['storekey'],
        row['productkey'],
        row['quantity'],
        row['currency_code'],
        row['shipping_days']
    ))

conn.commit()